<a href="https://colab.research.google.com/github/rongliyuan/ATAI-chatbot/blob/main/tutorials/streamlit_notebooks/T5_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/T5_SQL.ipynb)




### Spark NLP documentation and instructions:
https://nlp.johnsnowlabs.com/docs/en/quickstart

### Spark NLP Google T5 Article
https://towardsdatascience.com/hands-on-googles-text-to-text-transfer-transformer-t5-with-spark-nlp-6f7db75cecff

### For T5 models:
https://nlp.johnsnowlabs.com/models?q=T5

### You can find details about Spark NLP models here:
https://nlp.johnsnowlabs.com/models



## 1. Colab Setup

In [1]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 20.4 MB/s eta 0:00:00


# 2. Start the Spark session

In [2]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [3]:
spark = sparknlp.start()
print ("Spark NLP Version :", sparknlp.version())
spark

Spark NLP Version : 4.2.8


# Choose a sample text

In [38]:
text_list = [
            """How many customers have ordered more than 2 items?""",
             """How many players were with the school or club team La Salle?""",
             """When the scoring rank was 117, what was the best finish?""",
             """When the best finish was T69, how many people came in 2nd?""",
             """How many wins were there when the money list rank was 183?""",
             """When did the Metrostars have their first Rookie of the Year winner?""",
             """What college did the Rookie of the Year from the Columbus Crew attend?""",
            ]

### Detect Movie Entities - MIT Movie Simple

In [4]:
document_assembler = DocumentAssembler() \
.setInputCol('text') \
.setOutputCol('document')

tokenizer = Tokenizer() \
.setInputCols(['document']) \
.setOutputCol('token')

embeddings = DistilBertEmbeddings\
.pretrained('distilbert_base_cased', 'en')\
.setInputCols(["token", "document"])\
.setOutputCol("embeddings")

ner_model = NerDLModel.pretrained('ner_mit_movie_simple_distilbert_base_cased', 'en') \
.setInputCols(['document', 'token', 'embeddings']) \
.setOutputCol('ner')

ner_converter = NerConverter() \
.setInputCols(['document', 'token', 'ner']) \
.setOutputCol('entities')


distilbert_base_cased download started this may take some time.
Approximate size to download 232.7 MB
[OK!]
ner_mit_movie_simple_distilbert_base_cased download started this may take some time.
Approximate size to download 15.5 MB
[OK!]


In [5]:
ent_pipeline = Pipeline(stages=[
document_assembler,
tokenizer,
embeddings,
ner_model,
ner_converter
])

In [ ]:
# example = spark.createDataFrame(pd.DataFrame({'text': [
#     "Help me find a movie with the song when you really love a woman",
#     "What film featured Mel Gibson Danny Glover and Jet Li",
#     "Is there a documentary about the sports agency industry",

#     ]}))

In [6]:
example = spark.createDataFrame(pd.DataFrame({'text': [
             "What is the most current movie featuring Mat Damon",
              "What films use the song zippity do da",
              "In what genre is the movie Par 6",
              "Is there a documentary about the sports agency industry",
              "Channing Tatum has played what starring roles",
              "List of actors A Beautiful Mind",
             "Find the Melissa Leo Movie about smuggling illegals across the border",
             "Tell me the director, rating and publication date of a Apocalypse Now",
             "Avatar came out when and what did it gross",
             "Who is the director of Star Wars: Episode VI - Return of the Jedi? ",
             "Who is the screenwriter of The Masked Gang: Cyprus? ",
             "When was 'The Godfather' released? ",

]
}))

/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [7]:
result = ent_pipeline.fit(example).transform(example)

In [8]:
result.select("text", "entities").show(truncate=False)

+---------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                 |entities                                                                                                                                                                                                                       |
+---------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|What is the most current movie featuring Mat Damon                   |[{chunk, 41, 49, Mat Damon, {entity -> AC

In [9]:
ents = result.select("text", "entities").toPandas()

In [10]:
ents

,text,entities
0,What is the most current movie featuring Mat D...,"[(chunk, 41, 49, Mat Damon, {'sentence': '0', ..."
1,What films use the song zippity do da,"[(chunk, 24, 36, zippity do da, {'sentence': '..."
2,In what genre is the movie Par 6,"[(chunk, 27, 31, Par 6, {'sentence': '0', 'chu..."
3,Is there a documentary about the sports agency...,"[(chunk, 11, 21, documentary, {'sentence': '0'..."
4,Channing Tatum has played what starring roles,"[(chunk, 0, 13, Channing Tatum, {'sentence': '..."
5,List of actors A Beautiful Mind,"[(chunk, 17, 30, Beautiful Mind, {'sentence': ..."
6,Find the Melissa Leo Movie about smuggling ill...,"[(chunk, 9, 15, Melissa, {'sentence': '0', 'ch..."
7,"Tell me the director, rating and publication d...","[(chunk, 55, 68, Apocalypse Now, {'sentence': ..."
8,Avatar came out when and what did it gross,"[(chunk, 0, 5, Avatar, {'sentence': '0', 'chun..."
9,Who is the director of Star Wars: Episode VI -...,"[(chunk, 23, 64, Star Wars: Episode VI - Retur..."


In [72]:
result.select("ner").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# T5 for SQL Query generation

In [51]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")
t5 = T5Transformer.pretrained("t5_small_wikiSQL") \
    .setTask("translate English to SQL:") \
    .setInputCols(["documents"]) \
    .setMaxOutputLength(200) \
    .setOutputCol("sql")


t5_small_wikiSQL download started this may take some time.
Approximate size to download 249.9 MB
[OK!]


In [73]:
sql_pipeline = Pipeline().setStages([documentAssembler, t5])

In [50]:
# df = spark.createDataFrame(text_list, StringType()).toDF("text")
# result = sql_pipeline.fit(df).transform(df)

In [74]:
sql_result = sql_pipeline.fit(example).transform(example)

In [75]:
sql_result.select("text","sql.result").show(truncate=False)

+---------------------------------------------------------------------+------------------------------------------------------------------------------------------+
|text                                                                 |result                                                                                    |
+---------------------------------------------------------------------+------------------------------------------------------------------------------------------+
|What is the most current movie featuring Mat Damon                   |[SELECT MAX Current Movie FROM table WHERE Player = mat damon]                            |
|What films use the song zippity do da                                |[SELECT Films FROM table WHERE Song = zippity do da]                                      |
|In what genre is the movie Par 6                                     |[SELECT Genre FROM table WHERE Movie = par 6]                                             |
|Is there a documentar